# 3-Stage Clos Network with Port Channels
###### <sup>Inputs:  Bhavani Parise, Deepti Chandra; Developed by Sarah Samuel


The [3 Stage Clos Notebook](./3StageClos-Notebook.ipynb) walked you through the step-by-step procedure to configure a 3 stage Clos network on Cisco 8000 routers that run SONiC operating system. This notebook shows you how to configure a 3-stage Clos network with port channels.

The 3-stage Clos network is a robust IP-BGP underlay network that enables the servers to communicate with each other with minimum latency.

A port channel is a virtual interface on a network node such as a router or a switch that aggregate multiple physical interfaces into one logical interface.

When port channels are used instead of physical interfaces in a 3-stage Clos network, the network has the following benefits:
* Increased aggregate bandwidth of a link by distributing traffic among all functional members in the port channel.
* Since the port channel load balances the traffic across multiple member links, there is optimum bandwidth usage.
* Link redundancy - In case one member link fails, traffic previously carried on this link is switched to the remaining member links. If a member link in a port channel goes down, the upper protocols are not aware of it. Just the bandwidth is reduced. The MAC address tables are not affected by link failure either.

The following topology diagram depicts a simple 3-stage Clos network with port channels:

<center><img src="images/522667.jpg" width="700"/></center>


As you read through this notebook, play the code-cells, using the play button in the top left corner of this page, to send configuration commands to the live network nodes on the Cisco 8000 Emulator that runs in the background. The notebook refreshes the output of each cell-execution just beneath it.

[Access Device Consoles](#Access-Device-Consoles) and then play through the following steps to [Bring Up 3 Stage Clos Network with Port Channels](#Bring-Up-3-Stage-Clos-Network-with-Port-Channels):
* [Configure Host-Names](#Configure-Host-Names)
* [Set up Port Channels](#Set-up-Port-Channels)
* [Assign IP-Addresses](#Assign-IP-Addresses)
* [Configure eBGP](#Configure-eBGP)
* [Verify BGP Route Exchange](#Verify-BGP-Route-Exchange)
* [Send Traffic from TREX](#Send-Traffic-from-TREX)
* [Verify Traffic Statistics](#Verify-Traffic-Statistics)

Finally, [clean up emulator session](#Clean-Up-Emulator-Session), once you are done.

### Access Device Consoles

> Install prerequisite modules: This is required mainly for the first time you play this notebook. You can skip playing this cell for the subsequent runs.

In [1]:
%%capture cell-output
!sh lib/prereq_install.sh

> Play the following cell to bring up topology and access ssh console of each device in the topology. This cell takes about 10 - 15 minutes to complete execution. When you see the **Sim status** displayed as  **{'localhost': 'running'}** underneath this cell, the topology is up. Wait for a few more seconds until the code accesses the ssh console of each device. 

> Avoid playing this cell more than once, without [cleaning up the emulator session](#Clean-Up-Emulator-Session).

In [ ]:
from lib.leaf_spine import *
nodes = {
         'S0':'', 
         'S1':'',
         'L0':'', 
         'L1':'', 
         'trex':''
        }

tb = access_device_consoles("lib/leaf_spine.yaml", nodes)

## Bring Up 3-Stage Clos Network with Port Channels

> To configure a 3-stage Clos network, continue playing through the following steps:

### Configure Host-Names

> Configure host names for the spine and leaf routers for easy identification.

In [ ]:
out = nodes['S0'].execute('sudo config hostname SPINE0')
out = nodes['S0'].execute('sudo config save -y')
out = nodes['S1'].execute('sudo config hostname SPINE1')
out = nodes['S1'].execute('sudo config save -y')
out = nodes['L0'].execute('sudo config hostname LEAF0')
out = nodes['L0'].execute('sudo config save -y')
out = nodes['L1'].execute('sudo config hostname LEAF1')
out = nodes['L1'].execute('sudo config save -y')

### Set up Port Channels
> In this section, we will see how to configure regular port channels. Optionally, you can also
* [Set up Port Channels with Min-Links](#Set-up-Port-Channels-with-Min-Links)
* [Set up Port Channels with Fallback Option](#Set-up-Port-Channels-with-Fallback-Option)

> Configure port channels and add members to the port channel.

In [ ]:
print ("\n******************** Configuring S0 *************************")
out = nodes['S0'].execute('sudo config portchannel add PortChannel1')
out = nodes['S0'].execute('sudo config portchannel member add PortChannel1 Ethernet0')
out = nodes['S0'].execute('sudo config portchannel member add PortChannel1 Ethernet4')
out = nodes['S0'].execute('sudo config portchannel add PortChannel2')
out = nodes['S0'].execute('sudo config portchannel member add PortChannel2 Ethernet8')
out = nodes['S0'].execute('sudo config portchannel member add PortChannel2 Ethernet12')
print ("\n******************** Configuring S1 *************************")
out = nodes['S1'].execute('sudo config portchannel add PortChannel1')
out = nodes['S1'].execute('sudo config portchannel member add PortChannel1 Ethernet0')
out = nodes['S1'].execute('sudo config portchannel member add PortChannel1 Ethernet4')
out = nodes['S1'].execute('sudo config portchannel add PortChannel2')
out = nodes['S1'].execute('sudo config portchannel member add PortChannel2 Ethernet8')
out = nodes['S1'].execute('sudo config portchannel member add PortChannel2 Ethernet12')
print ("\n******************** Configuring L0 *************************")
out = nodes['L0'].execute('sudo config portchannel add PortChannel1')
out = nodes['L0'].execute('sudo config portchannel member add PortChannel1 Ethernet0')
out = nodes['L0'].execute('sudo config portchannel member add PortChannel1 Ethernet4')
out = nodes['L0'].execute('sudo config portchannel add PortChannel2')
out = nodes['L0'].execute('sudo config portchannel member add PortChannel2 Ethernet12')
out = nodes['L0'].execute('sudo config portchannel member add PortChannel2 Ethernet16')
print ("\n******************** Configuring L1 *************************")
out = nodes['L1'].execute('sudo config portchannel add PortChannel1')
out = nodes['L1'].execute('sudo config portchannel member add PortChannel1 Ethernet0')
out = nodes['L1'].execute('sudo config portchannel member add PortChannel1 Ethernet4')
out = nodes['L1'].execute('sudo config portchannel add PortChannel2')
out = nodes['L1'].execute('sudo config portchannel member add PortChannel2 Ethernet12')
out = nodes['L1'].execute('sudo config portchannel member add PortChannel2 Ethernet16')

> Verify the port channel configurations on all routers.

In [ ]:
for n in nodes:
   if (n != 'trex'):
      print ("\n******************** Command Outputs from", n, "*************************")
      out = nodes[n].execute('show interfaces portchannel')
      out = nodes[n].execute('show interface status Ethernet0-16')   
      out = nodes[n].execute('show interfaces status PortChannel1-2')

#### Set up Port Channels with Min-Links
> To ensure the port channel does not remain **Up**, if minimum number of member-links are not **Up**, configure the **min-links** option while configuring the port channel. This is an optional configuration. In the below cell, we will configure PortChannel1 of SPINE0 router with **min-links** of 2. 
> First, delete the existing port channel configurations and then reconfigure with min-links option.

In [ ]:
# Deleting the existing port channel configurations 
print ("\n** Deleting port channel configurations on S0 **")
out = nodes['S0'].execute('sudo config portchannel member del PortChannel1 Ethernet0')
out = nodes['S0'].execute('sudo config portchannel member del PortChannel1 Ethernet4')
out = nodes['S0'].execute('sudo config portchannel del PortChannel1')

# Configure port channel with min-links of 2
print ("\n** Configure port channel with min-links of 2 **")
out = nodes['S0'].execute('sudo config portchannel add PortChannel1 --min-links 2')
out = nodes['S0'].execute('sudo config portchannel member add PortChannel1 Ethernet0')
out = nodes['S0'].execute('sudo config portchannel member add PortChannel1 Ethernet4')

# Verify the port channel 
print ("\n** Verify the port channel **")
out = nodes['S0'].execute('show interface status Ethernet0-16')   
out = nodes['S0'].execute('show interfaces status PortChannel1')

> We can verify if min-links works as expected. If we shutdown one of the member-links of PortChannel1 i.e. Ethernet0, the **Oper** state of the port channel should be **down** in the output of ```show interfaces status PortChannel1``` 

In [ ]:
print ("\n** Verify that port channel is down when minimum number of member links (ie 2) are not Up **")
out = nodes['S0'].execute('sudo config interface shutdown Ethernet0')

# Verify that port channel is down when minimum number of member links (ie 2) are not Up
out = nodes['S0'].execute('show interface status Ethernet0-16')   
out = nodes['S0'].execute('show interfaces status PortChannel1')

> The port channel should come back to **up** state when the member link is unshut.

In [ ]:
# Bring up the member link Ethernet0
print ("\n** Verify that port channel is up when the member link is Up again **")
out = nodes['S0'].execute('sudo config interface startup Ethernet0')

# Verify that port channel is up when the member link is Up again
out = nodes['S0'].execute('show interface status Ethernet0-16')   
out = nodes['S0'].execute('show interfaces status PortChannel1')

#### Set up Port Channels with Fallback Option
> The fallback option allows an active member link of the port channel to establish the Link Aggregation (LAG) or port channel before receiving Link Aggregation Control Protocol (LACP) Protocol Data Unit (PDU) from its peer. LACP fallback will run on those member-links that have the lower port number. It runs on only one member port. This is an optional configuration. By default, the fallback option is set to **false**
> In the below cell, we will configure PortChannel1 of SPINE0 router with fallback option.

> First, shutdown Portchannel1 on LEAF0 so that it does not send any LACP PDUs to PortChannel1 on SPINE0. So the PortChannel1 on SPINE0 will be **down**.

In [ ]:
# Shutting down the port channel on L0
print ("\n** Shutting down the port channel on L0 **")
out = nodes['L0'].execute('sudo config interface shutdown PortChannel1')
out = nodes['L0'].execute('show interfaces status PortChannel1')

# The portchannel will be down on S0
print ("\n** The portchannel will be down on S0 **")
out = nodes['S0'].execute('show interfaces status PortChannel1')


> Delete the existing PortChannel1 configurations on SPINE0 and then reconfigure the PortChannel1 on SPINE0 with fallback option set to **true**.

> You will see that the PortChannel1 on SPINE0 is **Up** even though it does not receive any LACP PDUs from PortChannel1 on LEAF0 router. This confirms that fallback option works.

In [ ]:
print ("\n** Delete existing port channel on S0 **")
out = nodes['S0'].execute('sudo config portchannel member del PortChannel1 Ethernet0')
out = nodes['S0'].execute('sudo config portchannel member del PortChannel1 Ethernet4')
out = nodes['S0'].execute('sudo config portchannel del PortChannel1')

print ("\n** Configure port channel with fallback option set to true **")
out = nodes['S0'].execute('sudo config portchannel add PortChannel1 --fallback=true')
out = nodes['S0'].execute('sudo config portchannel member add PortChannel1 Ethernet0')
out = nodes['S0'].execute('sudo config portchannel member add PortChannel1 Ethernet4')

print ("\n** Verify the port channel on S0 is up **")
# Verify the port channel on S0 is up 
out = nodes['S0'].execute('show interface status Ethernet0-16')   
out = nodes['S0'].execute('show interfaces status PortChannel1')

> Finally unshut Portchannel1 on LEAF0. 

In [ ]:
# Unshut the port channel on L0
print ("\n** Unshut the port channel on L0 **")
out = nodes['L0'].execute('sudo config interface startup PortChannel1')
out = nodes['L0'].execute('show interfaces status PortChannel1')
out = nodes['S0'].execute('show interfaces status PortChannel1')

### Assign IP Addresses
> This step assigns IP addresses as per the topology diagram. And then saves the configurations.

In [ ]:
print ("\n******************** Configuring S0 *************************")
out = nodes['S0'].execute('sudo config interface ip add PortChannel1 10.0.1.1/24')
out = nodes['S0'].execute('sudo config interface ip add PortChannel2 10.0.2.1/24')
out = nodes['S0'].execute('sudo config interface ip add Loopback0 10.10.10.100/32')
out = nodes['S0'].execute('sudo config save -y')
print ("\n******************** Configuring S1 *************************")
out = nodes['S1'].execute('sudo config interface ip add PortChannel1 10.0.3.1/24')
out = nodes['S1'].execute('sudo config interface ip add PortChannel2 10.0.4.1/24')
out = nodes['S1'].execute('sudo config interface ip add Loopback0 10.10.11.100/32')
out = nodes['S1'].execute('sudo config save -y')
print ("\n******************** Configuring L0 *************************")
out = nodes['L0'].execute('sudo config interface ip add PortChannel1 10.0.1.2/24')
out = nodes['L0'].execute('sudo config interface ip add PortChannel2 10.0.3.2/24')
out = nodes['L0'].execute('sudo config interface ip add Ethernet8 10.0.5.1/24')
out = nodes['L0'].execute('sudo config interface ip add Loopback0 10.10.10.200/32')
out = nodes['L0'].execute('sudo config save -y')
print ("\n******************** Configuring L1 *************************")
out = nodes['L1'].execute('sudo config interface ip add PortChannel2 10.0.2.2/24')
out = nodes['L1'].execute('sudo config interface ip add PortChannel1 10.0.4.2/24')
out = nodes['L1'].execute('sudo config interface ip add Ethernet8 10.0.6.1/24')
out = nodes['L1'].execute('sudo config interface ip add Loopback0 10.10.11.200/32')
out = nodes['L1'].execute('sudo config save -y')
print ("\n******************** Configuring TREX *************************")
out = nodes['trex'].execute('ifconfig eth1 10.0.5.2 netmask 255.255.255.0 up')
out = nodes['trex'].execute('ifconfig eth2 10.0.6.2 netmask 255.255.255.0 up')

> Verify the configured IP addresses using the ```show ip interfaces``` command on the routers and ```ifconfig -a``` command on the TREX server.

In [ ]:
for n in nodes:
   print ("\n******************** Command Outputs from", n, "*************************")
   if (n == 'trex'):
      out = nodes[n].execute('ifconfig -a eth1')
      out = nodes[n].execute('ifconfig -a eth2')
   else:
      out = nodes[n].execute('show ip interfaces') 

### Configure eBGP

> The following section shows the eBGP (exterior Border Gateway Protocol) configuration on all spine and leaf routers. The spines are in Autonomous System (AS) 100 and the leaves in AS 200. 
> Note the usage of the ```allowas-in``` option in the ```neighbor``` command. The default behaviour of BGP is to reject learning routes from own AS. The ```allowas-in``` option overrides this default behavior and the nodes learn routes from own AS. 

In [ ]:

print ("\n******************** Configuring eBGP on S0 *************************")
out = nodes['S0'].execute ('''vtysh \
-c 'configure terminal' \
-c 'hostname SPINE_0' \
-c 'router-id 10.10.10.100' \
-c 'router bgp 100' \
-c 'no bgp ebgp-requires-policy' \
-c 'neighbor 10.0.2.2 remote-as 200' \
-c 'neighbor 10.0.1.2 remote-as 200' \
-c 'address-family ipv4 unicast' \
-c 'neighbor 10.0.2.2 allowas-in' \
-c 'neighbor 10.0.1.2 allowas-in' \
-c 'network 10.0.1.0/24' \
-c 'network 10.0.2.0/24' \
-c 'network 10.10.10.100/32' \
-c 'redistribute connected'
''')
print ("\n******************** Configuring eBGP on S1 *************************")
out = nodes['S1'].execute ('''vtysh \
-c 'configure terminal' \
-c 'hostname SPINE_1' \
-c 'router-id 10.10.11.100' \
-c 'router bgp 100' \
-c 'no bgp ebgp-requires-policy' \
-c 'neighbor 10.0.3.2 remote-as 200' \
-c 'neighbor 10.0.4.2 remote-as 200' \
-c 'address-family ipv4 unicast' \
-c 'neighbor 10.0.3.2 allowas-in' \
-c 'neighbor 10.0.4.2 allowas-in' \
-c 'network 10.0.3.0/24' \
-c 'network 10.0.4.0/24' \
-c 'network 10.10.11.100/32' \
-c 'redistribute connected'
''')
print ("\n******************** Configuring eBGP on L0 *************************")
out = nodes['L0'].execute ('''vtysh \
-c 'configure terminal' \
-c 'hostname LEAF_0' \
-c 'router-id 10.10.10.200' \
-c 'router bgp 200' \
-c 'no bgp ebgp-requires-policy' \
-c 'neighbor 10.0.1.1 remote-as 100' \
-c 'neighbor 10.0.3.1 remote-as 100' \
-c 'address-family ipv4 unicast' \
-c 'neighbor 10.0.1.1 allowas-in' \
-c 'neighbor 10.0.3.1 allowas-in' \
-c 'network 10.0.1.0/24' \
-c 'network 10.0.3.0/24' \
-c 'network 10.0.5.0/24' \
-c 'network 10.10.10.200/32' \
-c 'redistribute connected'
''')
print ("\n******************** Configuring eBGP on L1 *************************")
out = nodes['L1'].execute ('''vtysh \
-c 'configure terminal' \
-c 'hostname LEAF_1' \
-c 'router-id 10.10.11.200' \
-c 'router bgp 200' \
-c 'no bgp ebgp-requires-policy' \
-c 'neighbor 10.0.2.1 remote-as 100' \
-c 'neighbor 10.0.4.1 remote-as 100' \
-c 'address-family ipv4 unicast' \
-c 'neighbor 10.0.2.1 allowas-in' \
-c 'neighbor 10.0.4.1 allowas-in' \
-c 'network 10.0.2.0/24' \
-c 'network 10.0.4.0/24' \
-c 'network 10.0.6.0/24' \
-c 'network 10.10.11.200/32' \
-c 'redistribute connected'
''')

> Execute the command ```show ip bgp``` on all tier-0 and tier-1 routers to ensure eBGP is up.

In [ ]:
for n in nodes:
   if (n != 'trex'):
      print ("\n******************** Command Outputs from", n, "*************************")
      out = nodes[n].execute('''vtysh \
      -c 'show ip bgp'
      ''')

> This section checks the routes learnt through BGP by executing the command ```show ip route``` on all nodes in tier-0 and tier-1. The lines starting with the letter **B** are the routes learnt through BGP.

In [ ]:
for n in nodes:
   if (n != 'trex'):
      print ("\n******************** Command Outputs from", n, "*************************")
      out = nodes[n].execute('''vtysh \
      -c 'show ip route'
      ''')

> Ping IP addresses on the LEAF0 (L0) router from the LEAF1 (L1) router and vice-versa. This ensures that the 3-stage Clos network is ready for end-to-end traffic flow.

In [ ]:
print ("\n******************** Command Outputs from L1 to L0 *************************")
out = nodes['L1'].execute('ping -c5 10.0.5.1')
out = nodes['L1'].execute('ping -c5 10.10.10.200')
print ("\n******************** Command Outputs from L0 to L1 *************************")
out = nodes['L0'].execute('ping -c5 10.0.6.1')
out = nodes['L0'].execute('ping -c5 10.10.11.200')

**Pinging to remote leaf ip address is successful and so the 3-stage Clos network is up and ready for data traffic**

### Send Traffic from TREX

> [TREX](https://trex-tgn.cisco.com/trex/doc/trex_manual.html#_introduction) is a software traffic generator that runs on Linux. To simulate server-to-server traffic flow across a 3-stage Clos network, connect the TREX software traffic generator ports as the end-hosts to LEAF0 and LEAF1. 

> Details of traffic stream injected into LEAF0 from TREX:
> * Source IP address: 10.0.5.2
> * Destination IP address: 10.0.6.2

> Details of traffic stream injected into LEAF1 from TREX:
> * Source IP address: 10.0.6.2
> * Destination IP address: 10.0.5.2

<center><img src="./images/portchannel-traffic.png" width="700"/></center>


> The function ```generate_bidir_traffic``` generates a bidirectional traffic burst for 1 second. After the cell execution, check ```Total-tx-pkt``` and ```Total-rx-pkt``` in the ```summary stats``` at the end of the output to ensure that there is no traffic loss.

In [ ]:
trex_ipaddress = str(nodes['trex'].connections.cli.ip)
trex_port = str(nodes['trex'].connections.cli.port)

generate_bidir_traffic(trex_ipaddress, trex_port)

### Verify Traffic Statistics

> Check the interface counters on the nodes to ensure that the traffic is sent and received as seen in the traffic generator output.
    
> Note: "show interface counters rif" displaying 0s for all fields, is a known issue on the emulator. Users will experience expected behaviours on the physical setup.


In [ ]:
for n in nodes:
   if (n != 'trex'):
      out = nodes[n].execute('show interface counters rif')

> You have now successfully brought up a simple 3-stage Clos network, sent traffic across it. This forms the IP-BGP underlay for your data center network. 

> Scale up these configurations as per the number of devices in your data centre. 

### Clean Up Emulator Session

> Delete the ip addresses and port channels

In [ ]:
out = nodes['S0'].execute('sudo config interface ip remove PortChannel1 10.0.1.1/24')
out = nodes['S0'].execute('sudo config interface ip remove PortChannel2 10.0.2.1/24')
out = nodes['S0'].execute('sudo config interface ip remove Loopback0 10.10.10.100/32')
out = nodes['S1'].execute('sudo config interface ip remove PortChannel1 10.0.3.1/24')
out = nodes['S1'].execute('sudo config interface ip remove PortChannel2 10.0.4.1/24')
out = nodes['S1'].execute('sudo config interface ip remove Loopback0 10.10.11.100/32')
out = nodes['L0'].execute('sudo config interface ip remove PortChannel1 10.0.1.2/24')
out = nodes['L0'].execute('sudo config interface ip remove PortChannel2 10.0.3.2/24')
out = nodes['L0'].execute('sudo config interface ip remove Ethernet8 10.0.5.1/24')
out = nodes['L0'].execute('sudo config interface ip remove Loopback0 10.10.10.200/32')
out = nodes['L1'].execute('sudo config interface ip remove PortChannel2 10.0.2.2/24')
out = nodes['L1'].execute('sudo config interface ip remove PortChannel1 10.0.4.2/24')
out = nodes['L1'].execute('sudo config interface ip remove Ethernet8 10.0.6.1/24')
out = nodes['L1'].execute('sudo config interface ip remove Loopback0 10.10.11.200/32')

out = nodes['S0'].execute('sudo config portchannel member del PortChannel1 Ethernet0')
out = nodes['S0'].execute('sudo config portchannel member del PortChannel1 Ethernet4')
out = nodes['S0'].execute('sudo config portchannel del PortChannel1')
out = nodes['S0'].execute('sudo config portchannel member del PortChannel2 Ethernet8')
out = nodes['S0'].execute('sudo config portchannel member del PortChannel2 Ethernet12')
out = nodes['S0'].execute('sudo config portchannel del PortChannel2')
out = nodes['S1'].execute('sudo config portchannel member del PortChannel1 Ethernet0')
out = nodes['S1'].execute('sudo config portchannel member del PortChannel1 Ethernet4')
out = nodes['S1'].execute('sudo config portchannel del PortChannel1')
out = nodes['S1'].execute('sudo config portchannel member del PortChannel2 Ethernet8')
out = nodes['S1'].execute('sudo config portchannel member del PortChannel2 Ethernet12')
out = nodes['S1'].execute('sudo config portchannel del PortChannel2')
out = nodes['L0'].execute('sudo config portchannel member del PortChannel1 Ethernet0')
out = nodes['L0'].execute('sudo config portchannel member del PortChannel1 Ethernet4')
out = nodes['L0'].execute('sudo config portchannel del PortChannel1')
out = nodes['L0'].execute('sudo config portchannel member del PortChannel2 Ethernet12')
out = nodes['L0'].execute('sudo config portchannel member del PortChannel2 Ethernet16')
out = nodes['L0'].execute('sudo config portchannel del PortChannel2')
out = nodes['L1'].execute('sudo config portchannel member del PortChannel1 Ethernet0')
out = nodes['L1'].execute('sudo config portchannel member del PortChannel1 Ethernet4')
out = nodes['L1'].execute('sudo config portchannel del PortChannel1')
out = nodes['L1'].execute('sudo config portchannel member del PortChannel2 Ethernet12')
out = nodes['L1'].execute('sudo config portchannel member del PortChannel2 Ethernet16')
out = nodes['L1'].execute('sudo config portchannel del PortChannel2')

> Remove the router configurations so that you can play other notebooks

In [ ]:
# Remove the Router configs and release the interfaces used by trex
for n in nodes:
   if (n != 'trex'):
      out = nodes[n].execute('sudo rm /etc/sonic/config_db.json')
   else:
      out = nodes[n].execute('cd /opt/cisco/trex/latest/')
      out = nodes[n].execute('sudo ./dpdk_nic_bind.py --force -u 00:04.0')
      out = nodes[n].execute('sudo ./dpdk_nic_bind.py --force -u 00:05.0')
      out = nodes[n].execute('sudo ./dpdk_nic_bind.py --bind=virtio-pci 00:04.0')
      out = nodes[n].execute('sudo ./dpdk_nic_bind.py --bind=virtio-pci 00:05.0')

> Do let us know of your feedback or queries about this notebook at mig-notebooks@cisco.com.